See the end of this updated tools and agent notebook for how to use langchain memory as required in HW 4-2

In [1]:
%reload_ext dotenv
%dotenv -o

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.globals import set_debug
from langchain.agents import AgentExecutor

## Build an agent from scratch

https://python.langchain.com/docs/modules/agents/

In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Let's create a custom tool and include an explanation so the LLM can understand what it is useful for

In [4]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [5]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

How does the agent know what tools it can use? Those are passed in as a separate argument, so we can bind those as keyword arguments to the LLM.

In [6]:
llm_with_tools = llm.bind_tools(tools)

## Creating memory by hand

In order to do this, we need to do two things:

1. Add a place for memory variables to go in the prompt
2. Keep track of the chat history

First, let's add a place for memory in the prompt. We do this by adding a placeholder for messages with the key `chat_history`. Notice that we put this ABOVE the new user input (to follow the conversation flow).

In [7]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

We can then set up a list to track the chat history

In [8]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

Putting it together

In [9]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

When running, we now need to track the inputs and outputs as chat history

In [10]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


In [11]:
chat_history

[]

In [12]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a commonly used word in English. It may be a variation or abbreviation of the word "education."

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': '"Educa" is not a commonly used word in English. It may be a variation or abbreviation of the word "education."'}

In [13]:
chat_history

[HumanMessage(content='how many letters in the word educa?'),
 AIMessage(content='The word "educa" has 5 letters.')]

## Exercise 5-3: Using langchain's memory
**Exercise 5-3 (extra credit):** There are "memory" tools in langchain to manage this easier. Rewrite this example to use it

In [17]:
chat_history = []

In [18]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor_m = AgentExecutor(agent=agent, tools=tools, memory = memory, verbose=True)

The above pattern is so common that langchain provides a `ConversationBufferMemory` that does exactly what the above does (updating the `chat_history` key with the previous response) and the `AgentExecutor` constructor accepts a `memory` argument.

The `memory` argument is used [as follows](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html)
> Memory is a class that gets called at the start and at the end of every chain. At the start, memory loads variables and passes them along in the chain. At the end, it saves any returned variables. There are many different types of memory - please see memory docs for the full catalog.

In [21]:
memory.clear() # In case I am rerunning this cell
input1 = "how many letters in the word educe?"
result = agent_executor_m.invoke({"input": input1})
agent_executor_m.invoke({"input": "is that a real word?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educe'}`


5The word "educe" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "educe" is a real word. It means to bring out or develop (something potential or latent).

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educe?'),
  AIMessage(content='The word "educe" has 5 letters.'),
  HumanMessage(content='is that a real word?'),
  AIMessage(content='Yes, "educe" is a real word. It means to bring out or develop (something potential or latent).')],
 'output': 'Yes, "educe" is a real word. It means to bring out or develop (something potential or latent).'}

In [20]:
chat_history

[]